In [1]:
# Imprting libraries
import numpy as np # provides ways to work with large multidimensional arrays
import matplotlib.pyplot as plt # plotting
import yfinance as yf
import datetime as dt
import pandas as pd

# used to get data from a directory
import os
from os import listdir
from os.path import isfile, join

#### Dates and Other Constants

In [32]:
PATH = '/home/johnadi/Desktop/projects/projects-from-yt-channel-derek-banas/python-for-finance/Stock List/'

# Start Date Defaults
S_YEAR = 2017
S_MONTH = 1
S_DAY = 3
S_DATE_STR = "2017-01-03"
S_DATE_DATETIME = dt.datetime(S_YEAR, S_MONTH, S_DAY)

# End Date Defaults
E_YEAR = 2021
E_MONTH = 8
E_DAY = 18
E_DATE_STR = "2021-08-18"
E_DATE_DATETIME = dt.datetime(E_YEAR, E_MONTH, E_DAY)

#### Get Stock File Names in a list

In [3]:
files = [x for x in listdir(PATH) if isfile(join(PATH, x))]
tickers = [os.path.splitext(x)[0] for x in files]
tickers

['AIZ',
 'SYY',
 'PPL',
 'ETSY',
 'MTCH',
 'PARA',
 'HIG',
 'MMC',
 'IDXX',
 'NWSA',
 'ROP',
 'IT',
 'FDX',
 'J',
 'AES',
 'ZBH',
 'PYPL',
 'CVX',
 'GM',
 'MSFT',
 'TFC',
 'GEV',
 'KIM',
 'MU',
 'CBRE',
 'GE',
 'MTB',
 'PNR',
 'TXT',
 'AAL',
 'CMG',
 'LMT',
 'UPS',
 'MCK',
 'JKHY',
 'LDOS',
 'DAL',
 'QRVO',
 'DAY',
 'IVZ',
 'DGX',
 'CFG',
 'MPWR',
 'SPG',
 'JNJ',
 'SHW',
 'WMB',
 'FDS',
 'CPT',
 'GEHC',
 'PPG',
 'HES',
 'DPZ',
 'LKQ',
 'EBAY',
 'SRE',
 'RVTY',
 'EQR',
 'CI',
 'TFX',
 'CE',
 'BG',
 'COR',
 'MSCI',
 'STX',
 'FCX',
 'WMT',
 'GOOG',
 'TMUS',
 'JNPR',
 'KEYS',
 'VTRS',
 'FTNT',
 'FSLR',
 'ELV',
 'VRTX',
 'EQIX',
 'NEM',
 'TMO',
 'CEG',
 'ORLY',
 'TT',
 'BLDR',
 'IRM',
 'INCY',
 'STE',
 'MNST',
 'EXR',
 'EL',
 'EXC',
 'OKE',
 'EMN',
 'USB',
 'CNC',
 'CB',
 'WFC',
 'FOXA',
 'SYF',
 'HOLX',
 'LHX',
 'PG',
 'ADM',
 'ES',
 'AMD',
 'NUE',
 'TDY',
 'KMB',
 'HLT',
 'NI',
 'ROL',
 'APD',
 'TECH',
 'FMC',
 'GL',
 'ABBV',
 'FOX',
 'PAYX',
 'DLR',
 'RTX',
 'MPC',
 'RJF',
 'ESS',
 'VTR'

#### Function that returns a dataframe from a CSV

In [4]:
def get_df_from_csv(ticker):
    try:
        df = pd.read_csv(PATH + ticker + '.csv')
    except FileNotFoundError:
        pass
    else:
        return df

#### Function that saves dataframe to CSV

In [5]:
def save_dataframe_to_csv(df, ticker):
    df.to_csv(PATH + ticker + '.csv')

#### Return 1st valid date in dataframe

In [35]:
def get_valid_dates(df, sdate, edate):
    try:
        mask = (df['Date'] > sdate) & (df['Date'] <= edate)
        sm_df = df.loc[mask]
        sm_df = sm_df.set_index(['Date'])
        sm_date = sm_df.index.min()
        last_date = sm_df.index.max()
        date_leading = '-'.join(('0' if len(x)<2 else '')+x for x in sm_date.split('-'))
        date_ending = '-'.join(('0' if len(x)<2 else '')+x for x in last_date.split('-'))
    except Exception:
        print("Date Corrupted")
    else:
        return date_leading, date_ending

#### Returns Return on Investment over Time

In [7]:
def roi_between_dates(df, sdate, edate):
    try:
        start_val = df.loc[sdate, 'Adj Close']
        end_val = df.loc[edate, 'Adj Close']
        roi = ((end_val - start_val) / start_val)
    except Exception:
        print("Data Corrupted")
    else:
        return roi

#### Get MEAN between Dates

In [8]:
def get_mean_between_dates(df, sdate, edate):
    mask = (df['Date'] > sdate) & (df['Date'] <= edate)
    return df.loc[mask]['Adj Close'].mean()

#### Get Standard Deviation between Dates

In [9]:
def get_sd_between_dates(df, sdate, edate):
    mask = (df['Date'] > sdate) & (df['Date'] <= edate)
    return df.loc[mask]['Adj Close'].std()

#### Get Coefficient of Variation Between Dates

In [10]:
def get_cov_between_dates(df, sdate, edate):
    mean = get_mean_between_dates(df, sdate, edate)
    sd = get_sd_between_dates(df, sdate, edate)
    return sd/mean

#### Merge Multiple Stocks in One DataFrame by Column Name

In [11]:
def merge_df_by_column_name(col_name, sdate, edate, *tickers):
    # will hold data for all dataframes with the same column name
    mult_df = pd.DataFrame()

    for x in tickers:
        df = get_df_from_csv(x)
        df['Date'] = pd.to_datetime(df['Date'])
        # use a mask to grab data between defined dates
        mask = (df['Date'] >= sdate) & (df['Date'] <= edate)
        mult_df[x] = df.loc[mask][col_name]

    return mult_df

#### Stock Market Sectors

**Technology:** Manufacturing of electronics, software or related to information technology

**Health Care:** Biotech, hospitals, medical devices, drugs

**Financials:** Banks, investment funds, and insurance

**Real Estate:** Residential, industrial, and retail real estate

**Energy:** production and supply of energy

**Materials:** Mining, refining, chemical and forestry

**Consumer Discretionary:** Retailers, apparel, media, durables and services

**Industrials:** Construction, machinery, fabrication, manufacturing, defense, and aerospace

**Utilities:** Direct providers of electric, gas and water

**Consumer Staples:** Food, beverage as well as products consumers deem essential

**Telecommunication:** Cable, internet providers, wireless, and satellite

#### Get Sector Stocks

In [12]:
sec_df = pd.read_csv('./stock_sectors.csv')
sec_df

,Symbol,Name,Sector
0,MMM,3M,Industrials
1,AOS,A. O. Smith,Industrials
2,ABT,Abbott Laboratories,Health Care
3,ABBV,AbbVie,Health Care
4,ABMD,Abiomed,Health Care
...,...,...,...
500,YUM,Yum! Brands,Consumer Discretionary
501,ZBRA,Zebra Technologies,Information Technology
502,ZBH,Zimmer Biomet,Health Care
503,ZION,Zions Bancorp,Financials


In [13]:
indus_df = sec_df.loc[sec_df['Sector'] == "Industrials"]
indus_df

,Symbol,Name,Sector
0,MMM,3M,Industrials
1,AOS,A. O. Smith,Industrials
16,ALK,Alaska Air Group,Industrials
20,ALLE,Allegion,Industrials
29,AAL,American Airlines Group,Industrials
...,...,...,...
469,VRSK,Verisk Analytics,Industrials
479,GWW,W. W. Grainger,Industrials
480,WAB,Wabtec,Industrials
483,WM,Waste Management,Industrials


In [14]:
health_df = sec_df.loc[sec_df['Sector'] == "Health Care"]
it_df = sec_df.loc[sec_df['Sector'] == "Information Technology"]
comm_df = sec_df.loc[sec_df['Sector'] == "Communication Services"]
staple_df = sec_df.loc[sec_df['Sector'] == "Consumer Staples"]
discretion_df = sec_df.loc[sec_df['Sector'] == "Consumer Discretionary"]
utility_df = sec_df.loc[sec_df['Sector'] == "Utilities"]
financial_df = sec_df.loc[sec_df['Sector'] == "Financials"]
material_df = sec_df.loc[sec_df['Sector'] == "Materials"]
restate_df = sec_df.loc[sec_df['Sector'] == "Real Estate"]
energy_df = sec_df.loc[sec_df['Sector'] == "Energy"]

#### Returns a DF with ROIs for all Stocks

In [29]:
def get_rois_for_stocks(stock_df):
    tickers = []
    rois = []
    for index, row in stock_df.iterrows():
        df = get_df_from_csv(row['Symbol'])
        if df is None:
            pass
        else:
            tickers.append(row['Symbol'])
            sdate, edate = get_valid_dates(df, '2018-01-01', '2020-12-31')
            df = df.set_index(['Date'])
            roi = roi_between_dates(df, sdate, edate)
            rois.append(roi)
    return pd.DataFrame({'Ticker': tickers, 'ROI': rois})

In [37]:
industrials = get_rois_for_stocks(indus_df)
health_care = get_rois_for_stocks(health_df)
it = get_rois_for_stocks(it_df)
commun = get_rois_for_stocks(comm_df)
staple = get_rois_for_stocks(staple_df)
discretion = get_rois_for_stocks(discretion_df)
utility = get_rois_for_stocks(utility_df)
finance = get_rois_for_stocks(financial_df)
material = get_rois_for_stocks(material_df)
restate = get_rois_for_stocks(restate_df)
energy = get_rois_for_stocks(energy_df)

#### Top Industrials

In [39]:
industrials.sort_values(by=['ROI'], ascending=False).head()

,Ticker,ROI
22,GNRC,3.734749
9,CPRT,1.918578
57,TDG,1.518984
8,CTAS,1.325593
56,TT,1.231948


#### Top HealthCare

In [41]:
health_care.sort_values(by=['ROI'], ascending=False).head()

,Ticker,ROI
20,DXCM,5.375582
39,MRNA,4.616667
28,IDXX,2.167543
54,WST,1.887308
43,RMD,1.575014


#### Top Information Tech

In [42]:
it.sort_values(by=['ROI'], ascending=False).head()

,Ticker,ROI
19,ENPH,66.488464
2,AMD,7.352459
46,PAYC,4.622203
53,NOW,3.178471
64,ZBRA,2.705814


#### Top Communications

In [43]:
commun.sort_values(by=['ROI'], ascending=False).head()

,Ticker,ROI
10,NFLX,1.689262
14,TMUS,1.105715
3,CHTR,0.895722
15,TTWO,0.840804
9,LYV,0.722861


#### Top Staples

In [45]:
staple.sort_values(by=['ROI'], ascending=False).head()

,Ticker,ROI
10,EL,1.145197
9,COST,1.112863
19,MKC,0.977708
3,CHD,0.838448
25,PG,0.674050


#### Top Discretionary

In [46]:
discretion.sort_values(by=['ROI'], ascending=False).head()

,Ticker,ROI
46,TSLA,10.007861
17,ETSY,7.541047
10,CMG,3.733606
38,POOL,1.931029
45,TGT,1.839621


#### Top Utilities

In [47]:
utility.sort_values(by=['ROI'], ascending=False).head()

,Ticker,ROI
0,AES,1.402914
18,NEE,1.136332
4,AWK,0.819224
26,WEC,0.530405
27,XEL,0.519897


#### Top Finance

In [48]:
finance.sort_values(by=['ROI'], ascending=False).head()

,Ticker,ROI
37,MSCI,2.612069
32,MKTX,1.860333
6,AJG,1.112316
35,MCO,1.044883
46,SPGI,1.012123


#### Top Materials

In [49]:
material.sort_values(by=['ROI'], ascending=False).head()

,Ticker,ROI
22,SHW,0.824243
15,LIN,0.785526
0,APD,0.771644
18,NEM,0.689577
10,ECL,0.658577


#### Top Real Estate

In [50]:
restate.sort_values(by=['ROI'], ascending=False).head()

,Ticker,ROI
20,SBAC,0.772695
7,EQIX,0.692928
16,PLD,0.689758
1,AMT,0.685783
5,CCI,0.622156


#### Top Energy

In [51]:
energy.sort_values(by=['ROI'], ascending=False).head()

,Ticker,ROI
9,HES,0.168440
14,OKE,-0.131482
10,KMI,-0.146362
3,COP,-0.217487
19,WMB,-0.218709


#### Portfolio List

In [52]:
port_list = ['GNRC', 'DXCM', 'AMD', 'NFLX', 'COST', 'TGT', 'AES', 'MSCI', 'NEM', 'SBAC', 'HES']

#### Check for Correlation

In [53]:
mult_df = merge_df_by_column_name('daily_return', '2018-01-01', '2020-12-31', *port_list)
mult_df

,GNRC,DXCM,AMD,NFLX,COST,TGT,AES,MSCI,NEM,SBAC,HES
251,-0.030089,0.010455,0.068093,0.047458,0.011820,0.036475,0.004616,0.009009,0.017057,-0.016711,0.008005
252,0.035395,0.001552,0.051913,0.019794,0.012001,-0.006802,-0.000919,0.015194,-0.008386,0.012513,0.032393
253,0.006837,-0.100379,0.049351,0.002829,-0.007766,-0.019651,-0.003680,0.015738,0.011099,-0.005472,0.024089
254,0.006790,0.019330,-0.019802,0.021203,-0.007139,0.010630,0.003693,0.010405,0.003659,0.006925,0.031034
255,-0.017457,0.040931,0.033670,0.009810,0.003888,0.009467,0.000000,0.001353,-0.001042,0.000123,0.006327
...,...,...,...,...,...,...,...,...,...,...,...
1002,0.027268,-0.002725,0.002840,-0.000991,0.007433,0.001601,-0.009366,0.008935,0.011710,0.007473,-0.012177
1003,-0.000686,0.020534,-0.002287,0.010020,0.017774,0.006850,0.001289,0.009041,-0.010913,0.004473,-0.013261
1004,-0.050077,-0.014932,-0.010699,0.022634,0.004474,-0.007654,0.000429,-0.004125,-0.001504,-0.000905,-0.004922
1005,0.035326,0.016447,0.018429,-0.011830,0.004641,-0.006627,-0.022737,0.005569,0.011050,-0.005217,0.022827


In [54]:
mult_df.corr()

,GNRC,DXCM,AMD,NFLX,COST,TGT,AES,MSCI,NEM,SBAC,HES
GNRC,1.000000,0.289526,0.395187,0.340732,0.465304,0.338411,0.433323,0.515742,0.250697,0.444763,0.426870
DXCM,0.289526,1.000000,0.328297,0.307643,0.265366,0.196806,0.218424,0.469509,0.147493,0.243462,0.243239
AMD,0.395187,0.328297,1.000000,0.459581,0.394314,0.340573,0.258974,0.485819,0.191354,0.294906,0.288337
NFLX,0.340732,0.307643,0.459581,1.000000,0.430675,0.259971,0.204157,0.449132,0.103639,0.216935,0.217856
COST,0.465304,0.265366,0.394314,0.430675,1.000000,0.500434,0.319885,0.482323,0.183141,0.455022,0.234526
TGT,0.338411,0.196806,0.340573,0.259971,0.500434,1.000000,0.305004,0.325772,0.092756,0.304365,0.245105
AES,0.433323,0.218424,0.258974,0.204157,0.319885,0.305004,1.000000,0.483399,0.176915,0.333250,0.470412
MSCI,0.515742,0.469509,0.485819,0.449132,0.482323,0.325772,0.483399,1.000000,0.258931,0.498282,0.411647
NEM,0.250697,0.147493,0.191354,0.103639,0.183141,0.092756,0.176915,0.258931,1.000000,0.275682,0.149387
SBAC,0.444763,0.243462,0.294906,0.216935,0.455022,0.304365,0.333250,0.498282,0.275682,1.000000,0.273548


In [55]:
days = len(mult_df.index)
days

756

In [56]:
def get_stock_price_on_date(ticker, date):
    df = get_df_from_csv(ticker)
    df = df.set_index(['Date'])
    return df.loc[date, 'Adj Close']

In [57]:
print("GNRC $", get_stock_price_on_date("GNRC", '2021-01-04'))

GNRC $ 224.5599975585937


In [59]:
# 01-04-2021 is the 1st date in 2021
print("GNRC $", get_stock_price_on_date('GNRC', '2021-01-04')) # 2 449.12
print("DXCM $", get_stock_price_on_date('DXCM', '2021-01-04')) # 1 357.74
print("AMD $", get_stock_price_on_date('AMD', '2021-01-04')) # 4 369.2
print("NFLX $", get_stock_price_on_date('NFLX', '2021-01-04')) # 1 522.86
print("COST $", get_stock_price_on_date('COST', '2021-01-04')) # 1 377.88
print("TGT $", get_stock_price_on_date('TGT', '2021-01-04')) # 3 527.46
print("AES $", get_stock_price_on_date('AES', '2021-01-04')) # 18 410.76
print("MSCI $", get_stock_price_on_date('MSCI', '2021-01-04')) # 1 435.40
print("NEM $", get_stock_price_on_date('NEM', '2021-01-04')) # 7 434.49
print("SBAC $", get_stock_price_on_date('SBAC', '2021-01-04')) # 2 548.84
print("HES $", get_stock_price_on_date('HES', '2021-01-04')) # 9 472.23

# Total Investment
tot_inv = 449.12 + 357.74 + 369.2 + 522.86 + 377.88 + 527.46 + 410.76 + \
435.40 + 434.49 + 548.84 + 472.23
tot_inv

GNRC $ 224.5599975585937
DXCM $ 89.43499755859375
AMD $ 92.3000030517578
NFLX $ 522.8599853515625
COST $ 362.5612487792969
TGT $ 164.99349975585938
AES $ 20.939247131347656
MSCI $ 424.7478942871094
NEM $ 55.77037048339844
SBAC $ 265.93426513671875
HES $ 50.68441009521485


4905.980000000001

#### Returns Portfolio Weight

In [58]:
def get_port_weight(price, total):
    return price/total

In [60]:
gnrc_w = get_port_weight(449.12, 4905.98)
dxcm_w = get_port_weight(357.74, 4905.98)
amd_w = get_port_weight(369.2, 4905.98)
nflx_w = get_port_weight(522.86, 4905.98)
cost_w = get_port_weight(377.88, 4905.98)
tgt_w = get_port_weight(527.46, 4905.98)
aes_w = get_port_weight(410.76, 4905.98)
msci_w = get_port_weight(435.40, 4905.98)
nem_w = get_port_weight(434.49, 4905.98)
sbac_w = get_port_weight(548.84, 4905.98)
hes_w = get_port_weight(472.23, 4905.98)

wts = np.array([gnrc_w, dxcm_w, amd_w, nflx_w, cost_w, tgt_w, aes_w, msci_w, \
                nem_w, sbac_w, hes_w])
wts

port_var = np.dot(wts.T, np.dot(mult_df.cov() * 756, wts))
print("Portfolio Covariance :", port_var)
print("GNRC Var :", mult_df["GNRC"].var() * 756)
print("NFLX Var :", mult_df["NFLX"].var() * 756)

Portfolio Covariance : 0.18221024058431584
GNRC Var : 0.43946543607242516
NFLX Var : 0.5477253623043382


#### Different Types of Risk

There is risk that you can limit through diversification (idiosyncratic) and risk that you can't (Systematic). Systematic Risk is caused by unforeseen circumstances such as wars, recessions, natural disasters, dramatic changes in consumer actions etc. 

Data tells us that if we make a portfolio made up of approximately 25 stocks that are not correlated that we can dramatically lower idiosyncratic risk. There is one reason why people invest in indexes. You can further lower risk by investing in other countries, bonds and cash. 

#### Diversifiable Risk

In [61]:
# Diversifiable risk = portfolio variance - all weighted variance
def calc_diversifiable_risk(df, tickers, weights):
    days = len(mult_df.index)
    port_covar = np.dot(weights.T, np.dot(df.cov() * days, weights))
    i = 0
    while i < len(tickers):
        wt_sq = weights[i] ** 2
        stk_var = mult_df[tickers[i]].var() * days 
        wt_var = wt_sq * stk_var
        port_covar = port_covar - wt_var
        i += 1
    return port_covar

In [62]:
div_risk = calc_diversifiable_risk(mult_df, port_list, wts)
div_risk

0.1362664506932534

#### Find Risk that Can't be Diversified

In [63]:
print("Systematic Risk :", (port_var - div_risk))

Systematic Risk : 0.04594378989106243
